In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Choose a name for your CPU cluster
cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    cluster_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, cluster_config)

    cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = ### YOUR CODE HERE ###
    command_job(
        batch_size=Choice(values=[16, 32, 64, 128]),
        number_of_hidden_layers=Choice(values=range(1,5)),
    )
# Specify a Policy
policy = ### YOUR CODE HERE ###
         BanditPolicy(slack_factor = 0.1, delay_evaluation = 5, evaluation_interval = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ### YOUR CODE HERE ###
      ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = ### YOUR CODE HERE ###
                    HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='validation_acc',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4)

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = ### YOUR CODE HERE ###
     RandomParameterSampling(
        {
            '--batch-size': choice(25, 50, 100),
            '--first-layer-neurons': choice(10, 50, 200, 300, 500),
            '--second-layer-neurons': choice(10, 50, 200, 500),
            # '--learning-rate': loguniform(-6, -1)
        }
    )

# Specify a Policy
policy = ### YOUR CODE HERE ###
         BanditPolicy(slack_factor = 0.1, delay_evaluation = 5, evaluation_interval = 1)
    
if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                      script='./train.py',
                      compute_target=cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = ### YOUR CODE HERE ###
                    HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='validation_acc',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run_metric = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run_metric.get_file_names())
model_best_metric = best_run_metric.register_model(model_name='bankmarketing-best-output', model_path='outputs/model')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
#ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv', separator=',')
subscription_id = '510b94ba-e453-4417-988b-fbdc37b55ca7'
resource_group = 'aml-quickstarts-198208'
workspace_name = 'ml_DEMO'

workspace2 = Workspace(subscription_id, resource_group, workspace_name)
dataset = Dataset.get_by_name(workspace2, name='bankmarketing_train_dataset')
dataset.to_pandas_dataframe()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target = cluster,
    primary_metric='AUC_weighted',
    training_data=dataset,
    label_column_name='age',
    n_cross_validations= 5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
experiment = Experiment(ws, "bankmarketing-automl-run-2")
automl_run = experiment.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

model_best = best_run.register_model(model_name='bankmarketing-automl-output-2', model_path='outputs/model')